In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('.\data\LeagueofLegends.csv')
data.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,golddiff,...,redMiddleChamp,goldredMiddle,redADC,redADCChamp,goldredADC,redSupport,redSupportChamp,goldredSupport,redBans,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,"[0, 0, -14, -65, -268, -431, -488, -789, -494,...",...,Fizz,"[475, 475, 552, 842, 1178, 1378, 1635, 1949, 2...",Sneaky,Sivir,"[475, 475, 532, 762, 1097, 1469, 1726, 2112, 2...",LemonNation,Thresh,"[515, 515, 577, 722, 911, 1042, 1194, 1370, 14...","['Tristana', 'Leblanc', 'Nidalee']",http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...",...,Azir,"[475, 475, 552, 786, 1097, 1389, 1660, 1955, 2...",CoreJJ,Corki,"[475, 475, 532, 868, 1220, 1445, 1732, 1979, 2...",KiWiKiD,Annie,"[515, 515, 583, 752, 900, 1066, 1236, 1417, 15...","['RekSai', 'Janna', 'Leblanc']",http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...",...,Azir,"[475, 475, 533, 801, 1006, 1233, 1385, 1720, 1...",Cop,Corki,"[475, 475, 533, 781, 1085, 1398, 1782, 1957, 2...",BunnyFuFuu,Janna,"[515, 515, 584, 721, 858, 1002, 1168, 1303, 14...","['Leblanc', 'Zed', 'RekSai']",http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...",...,Lulu,"[475, 475, 532, 771, 1046, 1288, 1534, 1776, 2...",KEITH,KogMaw,"[475, 475, 532, 766, 1161, 1438, 1776, 1936, 2...",Xpecial,Janna,"[515, 515, 583, 721, 870, 1059, 1205, 1342, 15...","['RekSai', 'Rumble', 'LeeSin']",http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20...",...,Lulu,"[475, 475, 532, 807, 1042, 1338, 1646, 1951, 2...",Maplestreet8,Corki,"[475, 475, 532, 792, 1187, 1488, 1832, 2136, 2...",Dodo8,Annie,"[475, 475, 538, 671, 817, 948, 1104, 1240, 136...","['Rumble', 'Sivir', 'Rengar']",http://matchhistory.na.leagueoflegends.com/en/...


In [3]:
positions = ['Top', 'Jungle', 'Middle', 'ADC', 'Support']

regions = data['League'].unique()
major_regions = ['EULCS', 'LCK', 'NALCS', 'WC'] # EULCS - Europe, LCK - Korea, NALCS - US/Canada, WC - World Championship; unfortunately, China not included in the dataset
years = data['Year'].unique()
seasons = data['Season'].unique()
teams = list(set(data['blueTeamTag'].unique()).union(set(data['redTeamTag'].unique())))

In [63]:
cols = [team + pos + 'Champ' for team in ['blue', 'red'] for pos in positions]
data_condensed = data[data['Type'].isin(['International', 'Season'])][['League', 'Year', 'Season', 'blueTeamTag', 'bResult', 'rResult', 'redTeamTag', 'blueBans', 'redBans'] + cols]
data_condensed = data_condensed[data_condensed['League'].isin(major_regions)]
data_condensed.head()

,League,Year,Season,blueTeamTag,bResult,rResult,redTeamTag,blueBans,redBans,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp
0,NALCS,2015,Spring,TSM,1,0,C9,"['Rumble', 'Kassadin', 'Lissandra']","['Tristana', 'Leblanc', 'Nidalee']",Irelia,RekSai,Ahri,Jinx,Janna,Gnar,Elise,Fizz,Sivir,Thresh
1,NALCS,2015,Spring,CST,0,1,DIG,"['Kassadin', 'Sivir', 'Lissandra']","['RekSai', 'Janna', 'Leblanc']",Gnar,Rengar,Ahri,Caitlyn,Leona,Irelia,JarvanIV,Azir,Corki,Annie
2,NALCS,2015,Spring,WFX,1,0,GV,"['JarvanIV', 'Lissandra', 'Kassadin']","['Leblanc', 'Zed', 'RekSai']",Renekton,Rengar,Fizz,Sivir,Annie,Sion,LeeSin,Azir,Corki,Janna
3,NALCS,2015,Spring,TIP,0,1,TL,"['Annie', 'Lissandra', 'Kassadin']","['RekSai', 'Rumble', 'LeeSin']",Irelia,JarvanIV,Leblanc,Sivir,Thresh,Gnar,Nunu,Lulu,KogMaw,Janna
4,NALCS,2015,Spring,CLG,1,0,T8,"['Irelia', 'Pantheon', 'Kassadin']","['Rumble', 'Sivir', 'Rengar']",Gnar,JarvanIV,Lissandra,Tristana,Janna,Sion,RekSai,Lulu,Corki,Annie


In [70]:
# initialize a dictionary for each region, each of which will contain the counts for each champion
game_counts = {}
champ_counts = {}
for region in major_regions:
    for year in years:
        for season in seasons:
            champ_counts[(region, year, season)] = {}
            game_counts[(region, year, season)] = 0

# loop through rows, incrementing the respective counters for all champions in that game
for idx, row in data_condensed.iterrows():
    region, year, season = row[0:3]
    game_counts[(region, year, season)] += 1
    for champ in [x.strip() for x in row[7].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by blue
        if (champ, 'b') in champ_counts[(region, year, season)].keys():
            count, wins = champ_counts[(region, year, season)][(champ, 'b')]
            champ_counts[(region, year, season)][(champ, 'b')] = (count + 1, wins + row[4])
        else: # intialize new entry in dictionary, if necessary
            champ_counts[(region, year, season)][(champ, 'b')] = (1, row[4])
            
    for champ in row[9:14]: # for every champion present on blue
        if (champ, 'p') in champ_counts[(region, year, season)].keys():
            count, wins = champ_counts[(region, year, season)][(champ, 'p')]
            champ_counts[(region, year, season)][(champ, 'p')] = (count + 1, wins + row[4])
        else: # intialize new entry in dictionary, if necessary
            champ_counts[(region, year, season)][(champ, 'p')] = (1, row[4])
    
    for champ in [x.strip() for x in row[8].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by red
        if (champ, 'b') in champ_counts[(region, year, season)].keys():
            count, wins = champ_counts[(region, year, season)][(champ, 'b')]
            champ_counts[(region, year, season)][(champ, 'b')] = (count + 1, wins + row[5])
        else: # intialize new entry in dictionary, if necessary
            champ_counts[(region, year, season)][(champ, 'b')] = (1, row[5])
            
    for champ in row[14:]: # for every champion present on red
        if (champ, 'p') in champ_counts[(region, year, season)].keys():
            count, wins = champ_counts[(region, year, season)][(champ, 'p')]
            champ_counts[(region, year, season)][(champ, 'p')] = (count + 1, wins + row[5])
        else: # intialize new entry in dictionary, if necessary
            champ_counts[(region, year, season)][(champ, 'p')] = (1, row[5])

In [71]:
# for each region, create lists of champions and their counts, sorted by count
for region, counts in champ_counts.items():
    champ_list = np.array(list(counts.keys()))
    if len(champ_list) < 1:
        continue
    vals = np.array(list(counts.values()))
    count_vals = vals[:, 0]
    win_vals = vals[:, 1]
    idx = np.flip(np.argsort(count_vals))
    champ_list = champ_list[idx]
    count_vals = count_vals[idx]
    win_vals = win_vals[idx]
    
    print(region)
    for i in range(min(len(champ_list), 10)):
        print(champ_list[i], count_vals[i], win_vals[i])
    print()

('EULCS', 2015, 'Spring')
['Zed' 'b'] 49 23
['Graves' 'p'] 48 24
['Ahri' 'p'] 43 22
['JarvanIV' 'p'] 42 20
['Annie' 'p'] 40 19
['RekSai' 'p'] 39 16
['Kassadin' 'b'] 39 21
['Gnar' 'p'] 36 22
['Corki' 'p'] 36 13
['Cassiopeia' 'b'] 34 22

('EULCS', 2015, 'Summer')
['Gragas' 'p'] 60 34
['Alistar' 'p'] 53 25
['Ryze' 'b'] 52 22
['RekSai' 'p'] 51 25
['Kalista' 'b'] 47 19
['Sivir' 'p'] 40 22
['Corki' 'p'] 38 19
['Rumble' 'p'] 31 18
['Kalista' 'p'] 31 19
['Sivir' 'b'] 31 14

('EULCS', 2016, 'Spring')
['Lulu' 'b'] 65 27
['Gangplank' 'b'] 58 25
['Lucian' 'p'] 56 29
['Kalista' 'b'] 53 28
['Corki' 'p'] 50 27
['Braum' 'p'] 42 18
['Elise' 'p'] 41 17
['Alistar' 'p'] 40 20
['Poppy' 'p'] 34 17
['LeeSin' 'p'] 33 16

('EULCS', 2016, 'Summer')
['Nidalee' 'b'] 121 57
['Karma' 'p'] 100 41
['Vladimir' 'b'] 91 50
['RekSai' 'p'] 85 38
['Gragas' 'p'] 78 45
['Braum' 'p'] 78 42
['Sivir' 'p'] 77 36
['Trundle' 'p'] 73 36
['Viktor' 'p'] 72 30
['Elise' 'p'] 71 37

('EULCS', 2017, 'Spring')
['Camille' 'b'] 127 59
['Leb

In [72]:
s = 0.1
freq_champ_counts = {}
for region, counts in champ_counts.items():
    freq_champ_counts[region] = {}
    for champ, vals in counts.items():
        if vals[0]/game_counts[region] >= s:
            freq_champ_counts[region][champ] = vals

In [73]:
# initialize a dictionary for each region, each of which will contain the counts for each champion
champ_count_pairs = {}
for region in major_regions:
    for year in years:
        for season in seasons:
            champ_count_pairs[(region, year, season)] = {}

# loop through rows, incrementing the respective counters for all champions in that game
for idx, row in data_condensed.iterrows():
    region, year, season = row[0:3]
    b_basket = []
    r_basket = []
    
    for champ in [x.strip() for x in row[7].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by blue
        b_basket.append(champ)
    for champ in row[9:14]: # for every champion present on blue
        b_basket.append(champ)
    for champ in [x.strip() for x in row[8].replace("'", "").replace('[', '').replace(']', '').split(',')]: # for every champion banned by red
        r_basket.append(champ)
    for champ in row[14:]: # for every champion present on red
        r_basket.append(champ)
    
    for i in range(len(b_basket)):
        for j in range(i + 1, len(b_basket)):
            key = tuple(sorted([b_basket[i], b_basket[j]]) + ['same'])
            if key in champ_counts[(region, year, season)].keys():
                count, wins = champ_counts[(region, year, season)][key]
                champ_counts[(region, year, season)][key] = (count + 1, wins + row[4])
            else: # intialize new entry in dictionary, if necessary
                champ_counts[(region, year, season)][key] = (1, row[4])
    
    for i in range(len(r_basket)):
        for j in range(i + 1, len(r_basket)):
            key = tuple(sorted([r_basket[i], r_basket[j]]) + ['same'])
            if key in champ_counts[(region, year, season)].keys():
                count, wins = champ_counts[(region, year, season)][key]
                champ_counts[(region, year, season)][key] = (count + 1, wins + row[4])
            else: # intialize new entry in dictionary, if necessary
                champ_counts[(region, year, season)][key] = (1, row[4])

In [ ]:
eu_2015_spring = [123, 2, 32, ...] # aatrox, ahri, 